In [1]:
import nltk
import os
import re
import sklearn
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import BaggingClassifier

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,confusion_matrix

In [4]:
from sklearn.feature_selection import chi2,SelectKBest

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/u4512/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/u4512/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:
with open('Bookstrain.txt','r',encoding='utf-8') as f:
    book_train = f.readlines()

In [8]:
stopword = stopwords.words('english')

In [9]:
def preprocess(sentence):
    sentence = re.sub('[^\w\s]',' ',str(sentence))
    sentence = re.sub('[^a-zA-Z]',' ',str(sentence))
    new_sent = " "
    tok = word_tokenize(sentence)
    for i in range(len(tok)):
        if tok[i].lower() not in stopword:
            new_sent+=tok[i].lower()+" "
    return new_sent

In [10]:
corpus_train = []

In [11]:
label_train = np.zeros(1600)

In [12]:
label_train[0:800] = 1

In [13]:
for i in range(len(book_train)):
    sent = book_train[i]
    sent = sent[0:len(sent)-1]
    corpus_train.append(preprocess(sent))

In [14]:
len(corpus_train)

1600

In [15]:
len(label_train)

1600

In [16]:
with open('../Kitchen/Kitchentest.txt','r',encoding='utf-8') as f:
    kitchen_test = f.readlines()

In [17]:
corpus_test = []

In [18]:
label_test = np.zeros(400)

In [19]:
label_test[0:200] = 1

In [20]:
for i in range(400):
    sent = kitchen_test[i]
    sent = sent[0:len(sent)-1]
    corpus_test.append(preprocess(sent))

In [21]:
len(corpus_test)

400

In [22]:
len(label_test)

400

In [23]:
train_length = len(corpus_train)
test_length = len(corpus_test)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer(min_df = 3,max_df=0.8,use_idf=True,sublinear_tf=True,stop_words='english')

In [26]:
train_corpus_tf_idf = vectorizer.fit_transform(corpus_train)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [27]:
test_corpus_tf_idf = vectorizer.transform(corpus_test)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [28]:
train_corpus_tf_idf.shape

(1600, 6386)

In [29]:
vectorizer_chi2 = SelectKBest(chi2,k=4000)

In [30]:
chi_train_corpus_tf_idf = vectorizer_chi2.fit_transform(train_corpus_tf_idf,label_train)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [31]:
chi_test_corpus_tf_idf = vectorizer_chi2.transform(test_corpus_tf_idf)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [32]:
lr_classifier = LogisticRegression()
lr_classifier.fit(chi_train_corpus_tf_idf,label_train)
lr_pred = lr_classifier.predict(chi_test_corpus_tf_idf)
lr_acc = float((sum(lr_pred==label_test))/len(label_test))
lr_f1 = f1_score(lr_pred,label_test)
lr_cm = confusion_matrix(lr_pred,label_test)
print("The accuracy is :",lr_acc)
print("The f1 score is :",lr_f1)
print("confusion matrix is:")
print(lr_cm)

The accuracy is : 0.7275
The f1 score is : 0.7373493975903616
confusion matrix is:
[[138  47]
 [ 62 153]]


In [70]:
#Bagging Classifier with Logistic Regression
bg_classifier_lr = BaggingClassifier(base_estimator=lr_classifier,n_estimators=6,random_state=0,max_samples=0.5)
bg_classifier_lr.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_lr = bg_classifier_lr.predict(chi_test_corpus_tf_idf)
bg_accuracy_lr = bg_classifier_lr.score(chi_test_corpus_tf_idf,label_test)
bg_f1 = f1_score(bg_predict_lr,label_test)
bg_cm_lr = confusion_matrix(label_test,bg_predict_lr)
print("The accuracy is : ",bg_accuracy_lr)
print("The f1 score is :",bg_f1)
print("The confusion matrix is:")
print(bg_cm_lr)

The accuracy is :  0.73
The f1 score is : 0.75
The confusion matrix is:
[[130  70]
 [ 38 162]]


In [71]:
rbf_classifier = svm.SVC(gamma=0.9)
rbf_classifier.fit(chi_train_corpus_tf_idf,label_train)
rbf_pred = rbf_classifier.predict(chi_test_corpus_tf_idf)
rbf_accuracy = rbf_classifier.score(chi_test_corpus_tf_idf,label_test)
rbf_cm = confusion_matrix(rbf_pred,label_test)
rbf_f1 = f1_score(rbf_pred,label_test)
print("The SVM (rbf) Accuracy is: ",rbf_accuracy)
print("The F1 score is: ",rbf_f1)
print("The confusion matrix is:")
print(rbf_cm)

The SVM (rbf) Accuracy is:  0.72
The F1 score is:  0.7268292682926829
The confusion matrix is:
[[139  51]
 [ 61 149]]


In [72]:
#Bagging Classifier SVM with RBF Kernel
bg_classifier_rbf = BaggingClassifier(base_estimator=rbf_classifier,n_estimators=13,random_state=0,max_samples=0.7)
bg_classifier_rbf.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_rbf = bg_classifier_rbf.predict(chi_test_corpus_tf_idf)
bg_accuracy_rbf = bg_classifier_rbf.score(chi_test_corpus_tf_idf,label_test)
bg_cm_rbf = confusion_matrix(label_test,bg_predict_rbf)
bg_f1_rbf = f1_score(label_test,bg_predict_rbf)
print("The accuracy is : ",bg_accuracy_rbf)
print("The F1 score is: ",bg_f1_rbf)
print("The confusion matrix is:")
print(bg_cm_rbf)

The accuracy is :  0.7225
The F1 score is:  0.7312348668280871
The confusion matrix is:
[[138  62]
 [ 49 151]]


In [73]:
dt_classifier = DecisionTreeClassifier(random_state=0)
dt_classifier.fit(chi_train_corpus_tf_idf,label_train)
dt_pred = dt_classifier.predict(chi_test_corpus_tf_idf)
dt_accuracy = dt_classifier.score(chi_test_corpus_tf_idf,label_test)
dt_cm = confusion_matrix(dt_pred,label_test)
dt_f1 = f1_score(label_test,dt_pred)
print("The DT Accuracy is: ",dt_accuracy)
print("The F1 score is :",dt_f1)
print("The confusion matrix is:")
print(dt_cm)

The DT Accuracy is:  0.65
The F1 score is : 0.6391752577319586
The confusion matrix is:
[[136  76]
 [ 64 124]]


In [74]:
#Bagging with Decision Tree
bg_classifier_dt = BaggingClassifier(base_estimator=dt_classifier,n_estimators=15,random_state=0,max_samples=0.5)
bg_classifier_dt.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_dt = bg_classifier_dt.predict(chi_test_corpus_tf_idf)
bg_accuracy_dt = bg_classifier_dt.score(chi_test_corpus_tf_idf,label_test)
bg_cm_dt = confusion_matrix(label_test,bg_predict_dt)
bg_f1_dt = f1_score(label_test,bg_predict_dt)
print("The accuracy is: ",bg_accuracy_dt)
print("The F1 score is: ",bg_f1_dt)
print("The confusion matrix is:")
print(bg_cm_dt)

The accuracy is:  0.705
The F1 score is:  0.6589595375722543
The confusion matrix is:
[[168  32]
 [ 86 114]]


In [75]:
linear_classifier = svm.SVC(kernel='linear')
linear_classifier.fit(chi_train_corpus_tf_idf,label_train)
linear_pred = linear_classifier.predict(chi_test_corpus_tf_idf)
linear_accuracy = linear_classifier.score(chi_test_corpus_tf_idf,label_test)
linear_cm = confusion_matrix(linear_pred,label_test)
linear_f1 = f1_score(label_test,linear_pred)
print("The SVM(linear) Accuracy is: ",linear_accuracy)
print("The F1 score is :",linear_f1)
print("The confusion matrix is:")
print(linear_cm)

The SVM(linear) Accuracy is:  0.7
The F1 score is : 0.7
The confusion matrix is:
[[140  60]
 [ 60 140]]


In [76]:
#Bagging Classifier SVM with Linear Kernel
bg_classifier_linear = BaggingClassifier(base_estimator=linear_classifier,n_estimators=7,random_state=0,max_samples=0.9)
bg_classifier_linear.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_linear = bg_classifier_linear.predict(chi_test_corpus_tf_idf)
bg_accuracy_linear = bg_classifier_linear.score(chi_test_corpus_tf_idf,label_test)
bg_cm_linear = confusion_matrix(label_test,bg_predict_linear)
bg_f1_linear = f1_score(bg_predict_linear,label_test)
print("The accuracy is : ",bg_accuracy_linear)
print("The F1 score is :",bg_f1_linear)
print("The confusion matrix is:")
print(bg_cm_linear)

The accuracy is :  0.705
The F1 score is : 0.7107843137254902
The confusion matrix is:
[[137  63]
 [ 55 145]]


In [77]:
nb_classifier = MultinomialNB()
nb_classifier.fit(chi_train_corpus_tf_idf,label_train)
nb_acc = nb_classifier.score(chi_test_corpus_tf_idf,label_test)
nb_pred = nb_classifier.predict(chi_test_corpus_tf_idf)
nb_cm = confusion_matrix(nb_pred,label_test)
nb_f1 = f1_score(nb_pred,label_test)
print("Accuracy is :",nb_acc)
print("F1 score is :",nb_f1)
print("The confusion matrix is:")
print(nb_cm)

Accuracy is : 0.69
F1 score is : 0.6915422885572138
The confusion matrix is:
[[137  61]
 [ 63 139]]


In [78]:
#Bagging with Naive Bayes
bg_classifier_nb = BaggingClassifier(base_estimator=nb_classifier,n_estimators=5,random_state=0,max_samples=0.7)
bg_classifier_nb.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_nb = bg_classifier_nb.predict(chi_test_corpus_tf_idf)
bg_accuracy_nb = bg_classifier_nb.score(chi_test_corpus_tf_idf,label_test)
bg_cm_nb = confusion_matrix(label_test,bg_predict_nb)
bg_f1_nb = f1_score(bg_predict_nb,label_test)
print("The accuracy is: ",bg_accuracy_nb)
print("The F1 score is: ",bg_f1_nb)
print("The confusion matrix is:")
print(bg_cm_nb)

The accuracy is:  0.7
The F1 score is:  0.7142857142857143
The confusion matrix is:
[[130  70]
 [ 50 150]]


In [79]:
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(chi_train_corpus_tf_idf,label_train)
knn_pred = knn_classifier.predict(chi_test_corpus_tf_idf)
knn_accuracy = knn_classifier.score(chi_test_corpus_tf_idf,label_test)
knn_cm = confusion_matrix(knn_pred,label_test)
knn_f1 = f1_score(knn_pred,label_test)
print("Accuracy is :",knn_accuracy)
print("F1 score is :",knn_f1)
print("The confusion matrix is:")
print(knn_cm)

Accuracy is : 0.5175
F1 score is : 0.6655112651646448
The confusion matrix is:
[[ 15   8]
 [185 192]]


In [80]:
#Bagging with KNN
bg_classifier_knn = BaggingClassifier(base_estimator=knn_classifier,n_estimators=7,random_state=0,max_samples=0.5)
bg_classifier_knn.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_knn = bg_classifier_knn.predict(chi_test_corpus_tf_idf)
bg_accuracy_knn = bg_classifier_knn.score(chi_test_corpus_tf_idf,label_test)
bg_cm_knn = confusion_matrix(label_test,bg_predict_knn)
bg_f1_knn = f1_score(bg_predict_knn,label_test)
print("The accuracy is: ",bg_accuracy_knn)
print("The F1 score is: ",bg_f1_knn)
print("The confusion matrix is:")
print(bg_cm_knn)

The accuracy is:  0.5375
The F1 score is:  0.6748681898066784
The confusion matrix is:
[[ 23 177]
 [  8 192]]
